In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay  # from https://arxiv.org/pdf/1506.02078.pdf
from tensorflow.keras.callbacks import EarlyStopping

from tqdm.notebook import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
hidden_size = 128
seq_length = 200
learning_rate = 2e-3
dropout = 0.5
batch_size = 100
num_layers = 3
epochs = 50

In [4]:
data = open('mario_corpus.txt', 'r').read()
level_strs = data.rstrip().split(')')[:-1]
print(len(level_strs))

37


In [5]:
chars = []
for level_str in level_strs:
    chars.extend(list(level_str))
chars = list(set(chars))
vocab_size = len(chars)
print(chars, vocab_size)

['?', 'E', '>', ']', 'X', '-', 'b', 'x', 'o', '<', 'S', 'B', '[', 'Q', '\n'] 15


In [6]:
char_to_ix = { ch:i for i, ch in enumerate(chars) }
ix_to_char = { i:ch for i, ch in enumerate(chars) }

In [7]:
level_arrays = []
for level_str in level_strs:
    level_arrays.append(np.array([char_to_ix[char] for char in list(level_str)]))

In [8]:
def get_inputs_and_targets_from_level_array(level_array):
    
    inputs, targets = [], []
    
    for i in range(len(level_array) - seq_length):
        inputs.append(level_array[i:i+seq_length])
        targets.append(level_array[i+1:i+seq_length+1])
    
    inputs, targets = map(np.array, [inputs, targets])
    inputs = np.eye(vocab_size)[inputs]
    
    return inputs, targets

In [9]:
inputs, targets = [], []
for level_array in tqdm(level_arrays, leave=False):
    inputs_temp, targets_temp = get_inputs_and_targets_from_level_array(level_array)
    inputs.extend(inputs_temp); targets.extend(targets_temp)
inputs, targets = map(np.array, [inputs, targets])

In [10]:
inputs.shape, targets.shape

((119150, 200, 15), (119150, 200))

In [11]:
lr_scheduler = ExponentialDecay(
    initial_learning_rate=2e-3,
    decay_steps=len(inputs) // batch_size, 
    decay_rate=0.95, 
)
optimizer = RMSprop(learning_rate=lr_scheduler)

In [12]:
es_callback = EarlyStopping(
    monitor='val_loss', mode='min', patience=5, restore_best_weights=True
)

In [13]:
def custom_loss(y_true, y_pred):
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    return scce(
        tf.reshape(y_true, shape=(tf.shape(y_true)[0] * seq_length, )), 
        tf.reshape(y_pred, shape=(tf.shape(y_pred)[0] * seq_length, vocab_size))
    )

In [14]:
def custom_acc(y_true, y_pred):
    return tf.math.reduce_mean(
        tf.cast(
            tf.math.equal(
                tf.math.argmax(tf.reshape(y_pred, shape=(tf.shape(y_pred)[0] * seq_length, vocab_size)), axis=-1), 
                tf.cast(tf.reshape(y_true, shape=(tf.shape(y_true)[0] * seq_length, )), dtype=tf.int64)
            ), 
            dtype=tf.float32
        )
    )

In [51]:
model = keras.Sequential()

model.add(layers.LSTM(
    hidden_size, 
    return_sequences=True, 
))
model.add(layers.Dropout(dropout))

for i in range(num_layers - 1):

    model.add(layers.LSTM(
        hidden_size, 
        return_sequences=True, 
    ))
    model.add(layers.Dropout(dropout))

model.add(layers.Dense(vocab_size))
model.add(layers.Activation('softmax'))

model.compile(loss=custom_loss, optimizer=optimizer, metrics=[custom_acc])

In [52]:
history = model.fit(
    inputs, targets, 
    validation_split=0.1, 
    batch_size=batch_size, 
    shuffle=True, 
    epochs=50,
    callbacks=[es_callback]
)    

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1073/1073 [==============================] - 148s 138ms/step - loss: 0.4232 - custom_acc: 0.8754 - val_loss: 0.3709 - val_custom_acc: 0.8883
Epoch 2/50
1073/1073 [==============================] - 147s 137ms/step - loss: 0.1975 - custom_acc: 0.9410 - val_loss: 0.3150 - val_custom_acc: 0.9120
Epoch 3/50
1073/1073 [==============================] - 147s 137ms/step - loss: 0.1618 - custom_acc: 0.9514 - val_loss: 0.3648 - val_custom_acc: 0.9117
Epoch 4/50
1073/1073 [==============================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [54]:
# model.save('lstm_on_mario_corpus.h5')

In [55]:
model.evaluate(inputs, targets, verbose=1)

 330/3724 [=>............................] - ETA: 3:03 - loss: 0.1375 - custom_acc: 0.9561

KeyboardInterrupt: 

In [65]:
model = keras.models.load_model(
    'lstm_on_mario_corpus.h5', 
    compile=False
)

In [ ]:
model.evaluate(inputs, targets, verbose=1)

In [68]:
def onehot_to_string(onehot):
    ints = np.argmax(onehot, axis=-1)
    chars = [ix_to_char[ix] for ix in ints]
    string = "".join(chars)
    return string

In [80]:
seed = inputs[0][:200]

In [81]:
print(onehot_to_string(seed))

---------------X
---------------X
---------------X
---------------X
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
-------------xxX
-------------


In [82]:
gen = seed
for i in tqdm(range(1000)):
    probas = model.predict(
        np.expand_dims(seed, axis=0)
    )[0][-1]
    idx = np.random.choice(np.arange(len(probas)), p=probas)
    onehot_vector = np.zeros((1, vocab_size))
    onehot_vector[:, idx] = 1.
    seed = np.vstack([seed[1:], onehot_vector])
    gen = np.vstack([gen, onehot_vector])

In [83]:
print(onehot_to_string(gen))

---------------X
---------------X
---------------X
---------------X
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
-------------xxX
-------------xEX
-------------x-X
--------------xX
--------------xX
-------------BxX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
-------------xxX
-------------x--
-------------x-X
--------------xX
--------------xX
--------------xX
--------------xX
--------------xX
--------S-----xX
-------S---S--xX
------ES------xX
-------SE-----xX
------SS------xX
----S-SS---S--xX
----SSSSSSSS--xX
----SSSSSSSSS-xX
----SSSSSSSSS-xX
----SSSSSSSSSSxX
----SSSSSSSSSSxX
----SSSSSSSSS-xX
----S-S-SSSS--xX
----SSSSSSSSSSxX
----S-SSSSSSSSxX
----S-SSSSSSS-xX
----SSSSSSSSSSxX
----SSSSSSSSSSxX
----SSSSSSSSS-xX
----SSS---SSS-xX
----SSSSSSSSS-xX
----SSSSSSSSSSxX
----SSSSSSSSSSxX
----SSSSSSSSSS

In [ ]:
ans2 = model.predict()

In [73]:
ans == model.predict(ans).shape

/home/yangz2/anaconda3/envs/tf_with_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [22]:
dir(model.layers[0])

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_already_warned_about_input_casting',
 '_attribute_sentinel',
 '_auto_track_sub_layers',
 '_autocast',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_collect_input_masks',
 '_compute_dtype',
 '_could_use_gpu_kernel',
 '_create_dropout_mask',
 '_create_recurrent_dropout_mask',
 '_dedup_weights',
 '_deferr